<a href="https://colab.research.google.com/github/Snezhka111/Lab_7/blob/main/lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Встановлення та створення проєкту

Встановіть Scrapy в середовищі Python (pip install scrapy).

Створіть новий Scrapy-проєкт (scrapy startproject my_scraper).

Ознайомтеся зі структурою створеного проєкту (cd my_scraper && tree).

Запустіть Scrapy shell (scrapy shell "https://quotes.toscrape.com/"), щоб перевірити взаємодію з веб-сайтом.


In [ ]:
!pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 429.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 6.2 MB/s eta 0:00:00


In [ ]:
!scrapy startproject my_scraper

New Scrapy project 'my_scraper', using template directory '/usr/local/lib/python3.11/dist-packages/scrapy/templates/project', created in:
    /content/my_scraper

You can start your first spider with:
    cd my_scraper
    scrapy genspider example example.com


In [ ]:
!apt-get install tree
!tree my_scraper

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (86.5 kB/s)
Selecting previously unselected package tree.
(Reading database ... 126315 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
my_scraper
├── my_scraper
│   ├── __init__.py
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       └── __init__.py
└── scrapy.cfg

2 directories, 7 files


In [ ]:
import scrapy
from scrapy.http import Request
from scrapy.selector import Selector

import requests

response = requests.get("https://quotes.toscrape.com/")
selector = Selector(text=response.text)

quotes = selector.css("span.text::text").getall()
print(quotes)

['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”', '“It is our choices, Harry, that show what we truly are, far more than our abilities.”', '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”', '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”', "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”", '“Try not to become a man of success. Rather become a man of value.”', '“It is better to be hated for what you are than to be loved for what you are not.”', "“I have not failed. I've just found 10,000 ways that won't work.”", "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”", '“A day without sunshine is like, you know, night.”']


2. Створення базового Scrapy-павука

Створіть нового павука (spider) для отримання заголовків новин із сайту https://quotes.toscrape.com/.

Використайте XPath або CSS-селектори для вилучення заголовків цитат та їхніх авторів.

Налаштуйте павука так, щоб він обходив кілька сторінок (next page).

Запустіть Scrapy та виведіть отримані дані в консоль.


In [ ]:
import os
os.chdir('/content/my_scraper')

In [ ]:
!scrapy genspider quotes_spider quotes.toscrape.com

Created spider 'quotes_spider' using template 'basic' in module:
  my_scraper.spiders.quotes_spider


In [ ]:
import scrapy

class QuotesSpider(scrapy.Spider):
    name = "quotes_spider"
    start_urls = ['https://quotes.toscrape.com/']

    def parse(self, response):
        quotes = response.css('div.quote')

        for quote in quotes:
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get()
            }

        next_page = response.css('li.next a::attr(href)').get()
        if next_page:
            yield response.follow(next_page, callback=self.parse)


In [ ]:
!scrapy crawl quotes_spider

2025-04-15 20:39:11 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: my_scraper)
2025-04-15 20:39:11 [scrapy.utils.log] INFO: Versions: lxml 5.3.2.0, libxml2 2.12.10, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-04-15 20:39:11 [scrapy.addons] INFO: Enabled addons:
[]
2025-04-15 20:39:11 [asyncio] DEBUG: Using selector: EpollSelector
2025-04-15 20:39:11 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-04-15 20:39:11 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-04-15 20:39:11 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-04-15 20:39:11 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-04

3. Збереження даних

Збережіть результати роботи павука у файл CSV:

scrapy crawl myspider -o output.csv

Збережіть дані у форматах JSON та XML, порівняйте отримані файли.

In [ ]:
!scrapy crawl quotes_spider -o quotes.csv

2025-04-15 20:42:18 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: my_scraper)
2025-04-15 20:42:18 [scrapy.utils.log] INFO: Versions: lxml 5.3.2.0, libxml2 2.12.10, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-04-15 20:42:18 [scrapy.addons] INFO: Enabled addons:
[]
2025-04-15 20:42:18 [asyncio] DEBUG: Using selector: EpollSelector
2025-04-15 20:42:18 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-04-15 20:42:18 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-04-15 20:42:18 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-04-15 20:42:18 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-04

In [ ]:
!scrapy crawl quotes_spider -o quotes.json

2025-04-15 20:42:44 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: my_scraper)
2025-04-15 20:42:44 [scrapy.utils.log] INFO: Versions: lxml 5.3.2.0, libxml2 2.12.10, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-04-15 20:42:44 [scrapy.addons] INFO: Enabled addons:
[]
2025-04-15 20:42:44 [asyncio] DEBUG: Using selector: EpollSelector
2025-04-15 20:42:44 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-04-15 20:42:44 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-04-15 20:42:44 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-04-15 20:42:44 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-04

In [ ]:
!scrapy crawl quotes_spider -o quotes.xml

2025-04-15 20:43:10 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: my_scraper)
2025-04-15 20:43:10 [scrapy.utils.log] INFO: Versions: lxml 5.3.2.0, libxml2 2.12.10, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-04-15 20:43:10 [scrapy.addons] INFO: Enabled addons:
[]
2025-04-15 20:43:10 [asyncio] DEBUG: Using selector: EpollSelector
2025-04-15 20:43:10 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-04-15 20:43:10 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-04-15 20:43:10 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-04-15 20:43:10 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-04

Використання Scrapy Item & Pipelines

Створіть items.py для визначення полів (title, author, tags).

Змініть код павука так, щоб використовувати Scrapy Item.

Створіть pipelines.py, який буде очищувати дані

Додайте pipeline у settings.py, щоб активувати його.


In [ ]:
import scrapy

class QuoteItem(scrapy.Item):
    text = scrapy.Field()
    author = scrapy.Field()
    tags = scrapy.Field()

In [ ]:
import scrapy
from ..items import QuoteItem

class QuotesSpider(scrapy.Spider):
    name = "quotes_spider"
    start_urls = ['https://quotes.toscrape.com/']

    def parse(self, response):
        quotes = response.css('div.quote')

        for quote in quotes:
            item = QuoteItem()
            item['text'] = quote.css('span.text::text').get()
            item['author'] = quote.css('small.author::text').get()
            item['tags'] = quote.css('div.tags a.tag::text').getall()
            yield item

        next_page = response.css('li.next a::attr(href)').get()
        if next_page:
            yield response.follow(next_page, callback=self.parse)

In [ ]:
class QuotePipeline:
    def process_item(self, item, spider):
        # Прибираємо лапки з тексту цитати
        item['text'] = item['text'].strip('“”').strip()

        # Прибираємо пробіли в імені автора
        item['author'] = item['author'].strip()

        # Теги до нижнього регістру
        item['tags'] = [tag.lower() for tag in item['tags']]

        return item

In [ ]:
ITEM_PIPELINES = {
    'my_scraper.pipelines.QuotePipeline': 300,
}

In [ ]:
cd /path/to/my_scraper

[Errno 2] No such file or directory: '/path/to/my_scraper'
/content


In [ ]:
!scrapy crawl quotes_spider

2025-04-15 21:22:34 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: my_scraper)
2025-04-15 21:22:34 [scrapy.utils.log] INFO: Versions: lxml 5.3.2.0, libxml2 2.12.10, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-04-15 21:22:34 [scrapy.addons] INFO: Enabled addons:
[]
2025-04-15 21:22:34 [asyncio] DEBUG: Using selector: EpollSelector
2025-04-15 21:22:34 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-04-15 21:22:34 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-04-15 21:22:34 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2025-04-15 21:22:34 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2025-04

In [ ]:
import requests

# API ключ
api_key = '190c8cc898d38bd8e1e04dd4'

# Базова валюта, з якої ми конвертуємо
base_currency = 'USD'

# Валюти, на які ми хочемо отримати курс
target_currencies = ['EUR', 'GBP', 'UAH']

# Формуємо URL для запиту
url = f'https://v6.exchangerate-api.com/v6/{api_key}/latest/{base_currency}'

# Виконання запиту
response = requests.get(url)

# Перевірка статусу відповіді
if response.status_code == 200:
    data = response.json()

    # Якщо дані коректні
    if 'conversion_rates' in data:
        rates = data['conversion_rates']

        # Виведемо курси для вибраних валют
        for currency in target_currencies:
            print(f"1 {base_currency} = {rates[currency]} {currency}")
    else:
        print("Помилка: не вдалося отримати курси валют.")
else:
    print(f"Помилка: {response.status_code}")


1 USD = 0.8809 EUR
1 USD = 0.7592 GBP
1 USD = 41.3184 UAH
